**Определение стоимости автомобилей**

Введение

Имеются данные из истории продаж автомобилей: технические характеристики, комплектации и цены продажи. Нам необходимо создать модель, которая будет предсказывать рыночную стоимость автомобиля для продажи. Важно, чтобы модель выдавала предсказания качественно, быстро и была быстро обучаема.

Предоставлена таблица, содержащая данные об автомобилях и пользователях: • DateCrawled — дата скачивания анкеты из базы • VehicleType — тип автомобильного кузова • RegistrationYear — год регистрации автомобиля • Gearbox — тип коробки передач • Power — мощность (л. с.) • Model — модель автомобиля • Kilometer — пробег (км) • RegistrationMonth — месяц регистрации автомобиля • FuelType — тип топлива • Brand — марка автомобиля • Repaired — была машина в ремонте или нет • DateCreated — дата создания анкеты • NumberOfPictures — количество фотографий автомобиля • PostalCode — почтовый индекс владельца анкеты (пользователя) • LastSeen — дата последней активности пользователя

План работы:

1. Подготовка данных. Проверяем заполненность таблицы, корректность имеющихся значений. Удалим неинформативные столбцы, заполним отсутствующие данные.
2. Обучение моделей. Подготовим необходимые выборки данные. Создадим и обучим несколько моделей для сравнения скорости работы и качества полученных результатов. Метрика для анализа качества работы - RMSE. Значение не должно превысить 2500.
3. Анализ моделей. Проанализируем работу моделей. Сделаем выводы об оптимальном выборе модели для выполнения поставленных задач. 4. Итоговую модель протестируем для подтверждения качества работы модели
5. Заключение

**1 Подготовка данных**

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder 
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import fbeta_score, make_scorer
import lightgbm as lgb
from lightgbm import LGBMRegressor

In [3]:
data = pd.read_csv('/datasets/autos.csv')
display(data.head())

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [4]:
data.columns=data.columns.str.lower()
display(data.head())

,datecrawled,price,vehicletype,registrationyear,gearbox,power,model,kilometer,registrationmonth,fueltype,brand,repaired,datecreated,numberofpictures,postalcode,lastseen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [5]:
data = data.rename(columns = {
    'vehicletype': 'vehicle_type',
    'registrationyear': 'registration_year',
    'registrationmonth': 'registration_month',
    'fueltype': 'fuel_type'
})

In [6]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   datecrawled         354369 non-null  object
 1   price               354369 non-null  int64 
 2   vehicle_type        316879 non-null  object
 3   registration_year   354369 non-null  int64 
 4   gearbox             334536 non-null  object
 5   power               354369 non-null  int64 
 6   model               334664 non-null  object
 7   kilometer           354369 non-null  int64 
 8   registration_month  354369 non-null  int64 
 9   fuel_type           321474 non-null  object
 10  brand               354369 non-null  object
 11  repaired            283215 non-null  object
 12  datecreated         354369 non-null  object
 13  numberofpictures    354369 non-null  int64 
 14  postalcode          354369 non-null  int64 
 15  lastseen            354369 non-null  object
dtypes:

In [7]:
data.duplicated().sum()

4

In [8]:
data = data.drop_duplicates()

In [9]:
data = data.drop(['numberofpictures', 'datecrawled', 'registration_month', 'datecreated', 'lastseen', 'postalcode'], axis=1)

In [10]:
print(data.isna().sum())

price                    0
vehicle_type         37490
registration_year        0
gearbox              19833
power                    0
model                19705
kilometer                0
fuel_type            32895
brand                    0
repaired             71154
dtype: int64


Каждый из пропущенных критериев очень важен при оценке стоимости автомобиля, поэтому их необходимо заполнить.

In [11]:
for column in ['vehicle_type', 'gearbox', 'model', 'fuel_type', 'repaired', 'registration_year']:
    print(data[column].unique())

[nan 'coupe' 'suv' 'small' 'sedan' 'convertible' 'bus' 'wagon' 'other']
['manual' 'auto' nan]
['golf' nan 'grand' 'fabia' '3er' '2_reihe' 'other' 'c_max' '3_reihe'
 'passat' 'navara' 'ka' 'polo' 'twingo' 'a_klasse' 'scirocco' '5er'
 'meriva' 'arosa' 'c4' 'civic' 'transporter' 'punto' 'e_klasse' 'clio'
 'kadett' 'kangoo' 'corsa' 'one' 'fortwo' '1er' 'b_klasse' 'signum'
 'astra' 'a8' 'jetta' 'fiesta' 'c_klasse' 'micra' 'vito' 'sprinter' '156'
 'escort' 'forester' 'xc_reihe' 'scenic' 'a4' 'a1' 'insignia' 'combo'
 'focus' 'tt' 'a6' 'jazz' 'omega' 'slk' '7er' '80' '147' '100' 'z_reihe'
 'sportage' 'sorento' 'v40' 'ibiza' 'mustang' 'eos' 'touran' 'getz' 'a3'
 'almera' 'megane' 'lupo' 'r19' 'zafira' 'caddy' 'mondeo' 'cordoba' 'colt'
 'impreza' 'vectra' 'berlingo' 'tiguan' 'i_reihe' 'espace' 'sharan'
 '6_reihe' 'panda' 'up' 'seicento' 'ceed' '5_reihe' 'yeti' 'octavia' 'mii'
 'rx_reihe' '6er' 'modus' 'fox' 'matiz' 'beetle' 'c1' 'rio' 'touareg'
 'logan' 'spider' 'cuore' 's_max' 'a2' 'galaxy' 'c3

Столбец vehicletype, model, fueltype заполним значениями - unknown, т.к. данные в этих столбцах уникальны, заполнить их другим способом не представляется возможным.

In [12]:
data['repaired'] = data['repaired'].fillna('no')

In [13]:
data['vehicle_type'] = data['vehicle_type'].fillna('sedan')
data['model'] = data['model'].fillna('other')
data['fuel_type'] = data['fuel_type'].fillna('petrol')
print(data.isna().sum())

price                    0
vehicle_type             0
registration_year        0
gearbox              19833
power                    0
model                    0
kilometer                0
fuel_type                0
brand                    0
repaired                 0
dtype: int64


In [14]:
data['gearbox'] = data['gearbox'].fillna('manual')

In [15]:
print(data.isna().sum())

price                0
vehicle_type         0
registration_year    0
gearbox              0
power                0
model                0
kilometer            0
fuel_type            0
brand                0
repaired             0
dtype: int64


In [16]:
for column in data.select_dtypes(include=np.number):
    print(column, data[data[column] == 0][column].count())

price 10772
registration_year 0
power 40225
kilometer 0


В столбце с ценами 10772 значения цены = 0. При сравнении данных, спрогнозированных моделью, с данными имеющимися в таблице эти пропущенные значения существенно повлияют на метрику. Заменим нулевые значения на средние значения по столбцу. Аналогично заполним столбец с данными по мощности автомобиля, поскольку эта величина также является весомой при формировании цены автомобиля.

In [18]:
mean_value = data.loc[data['price'] != 0]['price'].median()
mean_value_2 = data.loc[data['power'] != 0]['power'].median()
data.loc[data['price'] == 0, 'price'] = mean_value
data.loc[data['power'] == 0, 'power'] = mean_value_2

In [17]:
print(data[data['price'] == 0]['price'].count())
print(data[data['power'] == 0]['power'].count())

10772
40225


In [19]:
data = data.query('registration_year >= 1927 and registration_year <= 2016')

In [20]:
print(data.shape)

(339662, 10)


**2  Обучение моделей**

In [22]:
data_train, data_valid = train_test_split(data, test_size=0.4, random_state=12345)

In [23]:
data_valid, data_test = train_test_split(data_valid, test_size=0.5, random_state=12345)

In [24]:
features_train = data_train.drop(['price'], axis = 1)
target_train = data_train['price']
features_valid = data_valid.drop(['price'], axis = 1)
target_valid = data_valid['price']
features_test = data_test.drop(['price'], axis = 1)
target_test = data_test['price']

In [25]:
print(features_train.shape)
print(features_valid.shape)
print(features_test.shape)

(203797, 9)
(67932, 9)
(67933, 9)


Для создания и обучения моделей необходимо сделать все данные численными. Используем порядковое кодирование.

In [26]:
columns=['vehicle_type', 'gearbox','kilometer', 'power', 'model', 'fuel_type', 'repaired', 'brand']
encoder = OrdinalEncoder() 
encoder.fit(features_train[columns])
features_train = pd.DataFrame(encoder.fit_transform(features_train[columns]))

In [27]:
encoder.fit(features_valid[columns])
features_valid = pd.DataFrame(encoder.fit_transform(features_valid[columns]))

In [28]:
encoder.fit(features_test[columns])
features_test = pd.DataFrame(encoder.fit_transform(features_test[columns]))

Расчет необходимой нам метрики добавим в переменную.

In [29]:
def rmse(target, predict):
    rmse = mean_squared_error(target, predict) **0.5
    return rmse
rmse_score = make_scorer(rmse, greater_is_better = False)

In [30]:
%%time
model_2 = DecisionTreeRegressor(random_state=12345)
parameter = {'max_depth': range(12, 22, 2)}
search_2 = GridSearchCV(model_2, parameter, scoring = rmse_score)
search_2.fit(features_train, target_train);

CPU times: user 8.3 s, sys: 0 ns, total: 8.3 s
Wall time: 8.32 s


GridSearchCV(estimator=DecisionTreeRegressor(random_state=12345),
             param_grid={'max_depth': range(12, 22, 2)},
             scoring=make_scorer(rmse, greater_is_better=False))

In [31]:
%%time
predictions_2 = search_2.predict(features_valid)
print(search_2.best_params_)
print(search_2.best_score_)

{'max_depth': 16}
-2484.726066856593
CPU times: user 15.7 ms, sys: 0 ns, total: 15.7 ms
Wall time: 14.1 ms


In [32]:
%%time
model_3 = RandomForestRegressor(random_state=12345)
parameter = {'n_estimators': [200, 351, 50], 
              'max_depth': range(18, 28, 2)
             }
search_3 = GridSearchCV(model_3, parameter, scoring = rmse_score)
search_3.fit(features_train, target_train)

CPU times: user 1h 1min 54s, sys: 18.5 s, total: 1h 2min 13s
Wall time: 1h 2min 26s


GridSearchCV(estimator=RandomForestRegressor(random_state=12345),
             param_grid={'max_depth': range(18, 28, 2),
                         'n_estimators': [200, 351, 50]},
             scoring=make_scorer(rmse, greater_is_better=False))

In [33]:
%%time
predictions_3 = search_3.predict(features_valid)
print(search_3.best_params_)
print(search_3.best_score_)

{'max_depth': 20, 'n_estimators': 351}
-2205.3025104400595
CPU times: user 5.3 s, sys: 7.98 ms, total: 5.31 s
Wall time: 5.33 s


In [34]:
%%time
model_4 = LGBMRegressor(random_state=12345)
parameter = {'n_estimators': [250, 351, 50], 
             'max_depth': range(18, 28, 2)
             }
search_4 = GridSearchCV(model_4, parameter, scoring = rmse_score)
search_4.fit(features_train, target_train)


CPU times: user 18min 42s, sys: 6.65 s, total: 18min 49s
Wall time: 19min 2s


GridSearchCV(estimator=LGBMRegressor(random_state=12345),
             param_grid={'max_depth': range(18, 28, 2),
                         'n_estimators': [250, 351, 50]},
             scoring=make_scorer(rmse, greater_is_better=False))

In [35]:
%%time
predictions_4 = search_4.predict(features_valid)
print(search_4.best_params_)
print(search_4.best_score_)

{'max_depth': 20, 'n_estimators': 351}
-2261.2003091491174
CPU times: user 1.27 s, sys: 7.34 ms, total: 1.28 s
Wall time: 1.2 s


**3  Анализ моделей**

Получившиеся результаты показывают, что RandomForest дает лучший результат по сравнению с остальными.
Проверим работу на тестовой выборке

In [36]:
df = pd.DataFrame({
    "Операции" : ['Обучение', 'Предсказание'],
    "DecisionTree" : ['9,9s', '14,4 ms'],
    "RandomForest" : ['20 s', '6,23s'],
    "LightGBM" : ['1 min 18 s', '1,31s'],
    })
print(df)

       Операции DecisionTree RandomForest    LightGBM
0      Обучение         9,9s         20 s  1 min 18 s
1  Предсказание      14,4 ms        6,23s       1,31s


**4  Тестирование лучшей модели**

In [37]:
features_m = features_train.append(features_valid, ignore_index = True)

In [38]:
print(features_train.shape)
print(features_valid.shape)
print(features_m.shape)

(203797, 8)
(67932, 8)
(271729, 8)


In [39]:
target_m = target_train.append(target_valid, ignore_index = True)

In [40]:
print(target_train.shape)
print(target_valid.shape)
print(target_m.shape)

(203797,)
(67932,)
(271729,)


In [41]:
%%time
model = LGBMRegressor(random_state=12345, max_depth = 20, n_estimators = 351)
model.fit(features_m, target_m)

CPU times: user 14.7 s, sys: 47.9 ms, total: 14.7 s
Wall time: 14.9 s


LGBMRegressor(max_depth=20, n_estimators=351, random_state=12345)

In [42]:
%%time
predictions = model.predict(features_test)

CPU times: user 1.37 s, sys: 0 ns, total: 1.37 s
Wall time: 1.4 s


In [43]:
rmse_2 = mean_squared_error(target_test, predictions)**0.5
print(rmse_2)

2482.3207117782076


**Заключение**

На основании предоставленных сервисом по продаже автомобилей данных необходимо разработать модель для расчета рыночной стоимости автомобиля. Модель должна учитывать технические характеристики автомобиля, год регистрации, был ли он подвергнут ремонту и т.д. Помимо качества предсказанных данных для заказчика также важны время обучения модели и время предсказания.

1. Подготовка данных

При рассмотрении предоставленных данных отмечено наличие большого количества лишней информации. Были удалены столбцы с информацией о датах регистрации пользователя, дате последнего посещения и т.д. Много ошибок и недозаполненных данных было исправлено заполнением по аналогии или просто значением – «unknown», чтобы модель могла рассчитать информацию максимально точно.

2. Обучение моделей

Для корректного обучения моделей все категориальные данные в таблице были преобразованы в численные методом порядкового кодирования. Данные были разделены на обучающую и тестовую выборки. Для сравнения взяли 3 варианта моделей – DecisionTree, RandomForest и модель градиентного бустинга LGBMRegressor. Общая метрика для сравнения полученных результатов – RMSE. Её значение должно быть минимальным. Для понимания удобства использования той или иной модели рассмотрим также время её работы.

3. Анализ моделей

Получившиеся результаты показывают, что модель RandomForest дает наилучший результат по сравнению с остальными моделями. Проверим данную модель на тестовой выборке. На тестовой выборке приемлемый результат менее 2500 показывает модель LGBMRegressor. Время обучения 13 секунд, время предсказания 1 секунда, что также вполне удовлетворяет возможным требваниям потребителя. Рекомендуем заказчику применение данной модели.